Optimize the model to get at least 70% accuray .you may play with

1. Dynamic learning rate
2. Save model
3. Early stopping
4. Drop-Out**
for getting higher accuracy

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
## import matplotlib.pyplot as plt
import glob
import librosa
import numpy as np
import cv2 
bleached_corals = glob.glob('/kaggle/input/healthy-and-bleached-corals-image-classification/bleached_corals/*.jpg')
healthy_corals = glob.glob('/kaggle/input/healthy-and-bleached-corals-image-classification/healthy_corals/*.jpg')

data = []
labels = []

image_size=(32,32)

for i in bleached_corals:   
    image = cv2.imread(i,cv2.IMREAD_GRAYSCALE)
    # Resize the image
    image = cv2.resize(image, image_size)

    data.append(image)
    labels.append(0)

for i in healthy_corals:   
    image = cv2.imread(i,cv2.IMREAD_GRAYSCALE)
    # Resize the image
    image = cv2.resize(image, image_size)

    data.append(image)
    labels.append(1)
    
    
X=np.array(data)
y=np.array(labels)
X.shape

In [ ]:
# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Perform train-validation split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
# Build the MLP model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(32,32)),
    
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define the learning rate reduction callback
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',   # Monitor validation loss for learning rate reduction
    factor=0.1,            # Reduce learning rate by a factor of 0.1
    patience=4,            # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=2e-7           # Minimum learning rate
)



# Define the model checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint('best_model2', monitor='val_accuracy', save_best_only=True, mode= 'max')

# Define the early stopping callback to stop training if validation loss does not improve
early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=5)

# Train the model with the learning rate reduction callback
model.fit(X_train, y_train,validation_data=(X_val,y_val),epochs=20, batch_size=8,callbacks=[lr_callback,checkpoint_callback,early_stopping_callback])

# Evaluate the model
best_model = tf.keras.models.load_model('best_model2')
loss, accuracy = best_model.evaluate(X_test, y_test)

print(loss, accuracy )
